In [1]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from json import load, dump as save
from datetime import datetime
from os import listdir
import re

clean = re.compile(r'\n|\s{2,}').sub
id = lambda item: item.get_attribute('data-id')
price = lambda item: int(item.find_element_by_xpath('.//span[@itemprop="price"]').text)
url = 'https://www.splashfashions.com/ae/en/search?price=0,40&q=allCategories:men-regular:badge.title.en:Sale:index:price'

In [2]:
def foreach_item(execute):
    for page in range(2, 22):
        [execute(item) for item in br.find_elements_by_class_name('product-item')]
        try: br.find_element_by_class_name('next').click()
        except: break
        if page < 21: WebDriverWait(br, 10).until(EC.text_to_be_present_in_element((By.CLASS_NAME, 'current'), str(page)))

def get_old_discounts():
    try: return load(open('old.json'))
    except: old = {}
    foreach_item(lambda item: old.setdefault(id(item), price(item)))
    save(old, open('old.json', 'w+'))
    return old

def check_new_discounts():
    old, new = get_old_discounts(), []
    def check_item(item):
        _id = id(item)
        _price = price(item)
        if _price < old.get(_id, 1000):
            new.append(clean('', item.get_attribute('outerHTML')))
            old[_id] = _price
    foreach_item(check_item)
    if new:
        save(old, open('old.json', 'w'))
        save(new, open(f"new_{str(datetime.now())[:-10].replace(' ', '_').replace(':', '-')}.json", 'w+'))
    return f'Found {len(new)} new discounts'

In [3]:
def show_new_discounts():
    items = []
    for file in listdir():
        if file.startswith('new') and file.endswith('json'):
            items.extend(load(open(file)))
    html = ''.join(items).replace("'", r"\'")
    br.execute_script(f"document.getElementById('products-list').innerHTML = '{html}';")
    br.execute_script(f"document.getElementById('sort-panel').remove();")
    br.execute_script(f"document.getElementById('lms-pagination').remove();")

### Run broweser and check for discounts

In [4]:
br = Chrome()
br.get(url)

In [5]:
check_new_discounts()

'Found 0 new discounts'

In [6]:
show_new_discounts()